In [ ]:
import warnings
from datetime import datetime

import cartopy.crs as ccrs
import iris
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from wildfires.analysis import cube_plotting
from wildfires.data import (
    Ext_MOD15A2H_fPAR,
    MOD15A2H_LAI_fPAR,
    dataset_times,
    dummy_lat_lon_cube,
)
from wildfires.logging_config import enable_logging
from wildfires.utils import ensure_datetime, get_land_mask, match_shape

warnings.filterwarnings("ignore", "Collapsing a non-contiguous coordinate.*")

enable_logging(mode="jupyter")

In [ ]:
def mask_water(cube):
    assert isinstance(cube.data, np.ndarray)
    if not hasattr(cube.data, "mask"):
        cube.data = np.ma.MaskedArray(
            cube.data, mask=np.zeros_like(cube.data, dtype=np.bool_)
        )
    cube.data.mask |= ~match_shape(get_land_mask(), cube.shape)
    return cube

In [ ]:
ext_mod = Ext_MOD15A2H_fPAR()
mod = MOD15A2H_LAI_fPAR()

In [ ]:
min_time, max_time, time_df = dataset_times((ext_mod, mod))
# Discard anything but the year and month.
min_time = datetime(min_time.year, min_time.month, 1)
max_time = datetime(max_time.year, max_time.month, 1)
time_df

In [ ]:
fapar_name = "Fraction of Absorbed Photosynthetically Active Radiation"

In [ ]:
mod_fapar = mod.get_monthly_data(start=min_time, end=max_time).extract_cube(
    iris.Constraint(fapar_name)
)
ext_mod_fapar = ext_mod.get_monthly_data(start=min_time, end=max_time).extract_cube(
    iris.Constraint(fapar_name)
)

In [ ]:
mod_fapar.units = "1"

In [ ]:
ext_mod_fapar

In [ ]:
mod_fapar

In [ ]:
fig_kwargs = dict(figsize=(17, 4), dpi=200)

In [ ]:
for i in tqdm(range(mod_fapar.shape[0])):
    fig, axes = plt.subplots(
        1, 3, subplot_kw=dict(projection=ccrs.Robinson()), **fig_kwargs
    )
    for ax, cube, title in zip(
        axes,
        (
            mod_fapar[i],
            ext_mod_fapar[i],
            dummy_lat_lon_cube(ext_mod_fapar[i].data - mod_fapar[i].data),
        ),
        ("MOD", "Ext MOD", "Ext MOD - MOD"),
    ):
        ax.set_title(title)
        kwargs = {}
        if "-" in title:
            kwargs = {
                **kwargs,
                **dict(
                    cmap="RdBu_r",
                    cmap_midpoint=0,
                    cmap_symmetric=True,
                    vmin_vmax_percentiles=(2, 98),
                ),
            }

        cube_plotting(
            mask_water(cube),
            ax=ax,
            title=f"{ensure_datetime(mod_fapar[i].coord('time').cell(0).point):%Y-%m}",
            **kwargs,
        )